In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyspark as psk  
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
from IPython import get_ipython


findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = (SparkConf().set("spark.driver.maxResultSize", "20g"))

# Create new context
sc = SparkContext(conf=conf)


# sc = SparkContext("local", "Simple App")

# Create SparkSession 
spark = SparkSession.builder \
       .master("local[2]") \
       .appName("test") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()

#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)  # Increase to 1000 or more as needed

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

df_population = spark.sql("SELECT * FROM population")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/15 23:07:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/03/15 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4045. Att

In [3]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

# Functions

In [4]:
'''
Data Normalization Using Z-score Method
The z-score method (often called standardization) transforms the info into distribution with a mean of 0 and a typical deviation of 1. 
Each standardized value is computed by subtracting the mean of the corresponding feature then dividing by the quality deviation.

One standard is to include an epsilon variable that prevents divide by zero. 
Epsilon is an intentional error added to calculations to prevent creating NaN or Inf.
Fact about z-score[column] = 0: mSince the standard deviation is calculated by taking 
the sum of the squared deviations from the mean, a zero standard deviation can only be possible 
when all the values of a variable are the same (all equal to the mean). 
In this case, those variables have no discriminative power so they can be removed from the analysis. 
They cannot improve any classification, clustering or regression task. 
Many implementations will do it for you or throw an error about a matrix calculation.
'''
def normalize_numerical_data(df_numeric_columns):
   #first 2 columns, ENGIN_HP and calendar_day are not normalized and neither the latest two columns (target calumns), parts_are_replaced columns
    number_of_numeric_features = len(df_numeric_columns.columns)-2
    epsilon=1e-100
    for column in df_numeric_columns.iloc[:,2:number_of_numeric_features].columns:
        df_numeric_columns[column] = (df_numeric_columns[column] -
                               df_numeric_columns[column].mean()) / (df_numeric_columns[column].std()+epsilon)

    return df_numeric_columns



def normalize_numeric_feature_values(statement_in_feature_name, df):

  selected_col_names_list = [col for col in df.columns.values if statement_in_feature_name in col]   # selects names of columns that contain specific string

  selected_cols = df[selected_col_names_list]
  # for col_name in selected_col_names_list:
      # selected_cols[col_name] = selected_cols[col_name].str.replace(",",".")
      # selected_cols[col_name] = selected_cols[col_name].apply(lambda x: float(x.split()[0]))
  selected_cols_norm = selected_cols.apply(lambda iterator: ((iterator - iterator.mean())/(iterator.max() - iterator.min())).round(3))

  df[selected_cols_norm.columns] = selected_cols_norm
  return df


def check_non_empty_string(input_string):
    if not isinstance(input_string, str):
        return False
    
    if input_string.strip() == "":
        return False
    
    return True



'''
Perform z-score normalization by subtracting the mean and dividing by the standard deviation (or the replaced value).

'''
def normalize_sparse_dataframe(df, skip_this_col):
    """
    Normalize a DataFrame with float numeric values, ensuring no NaN values are produced.
    """
    original_df = df
    columns_to_normalize = df.columns
    
    # First exclude this column (if necessary)
    if check_non_empty_string(skip_this_col):
        columns_to_normalize = [col for col in df.columns if skip_this_col not in col]
        df = df[columns_to_normalize]
        
    # Calculate the mean and standard deviation for each column
    mean = df.mean()
    std = df.std()
    
    # Avoid division by zero by setting std deviations of 0 to 1 (no scaling for those columns)
    std_replaced = std.replace(0, 1)
    
    # Perform the normalization
    normalized_df = (df - mean) / std_replaced
    original_df[columns_to_normalize] = normalized_df
    
    return original_df



def preprocess_csv(file_path, fixed_file_path, expected_columns):
    """
    Preprocess the CSV file to fix bad lines.

    Parameters:
    - file_path: Path to the input CSV file.
    - fixed_file_path: Path to save the fixed CSV file.
    - expected_columns: Expected number of columns in the CSV file.
    """
    with open(file_path, 'r', encoding='utf-8') as infile, open(fixed_file_path, 'w', encoding='utf-8') as outfile:
        buffer = ""
        line_count = 0
        skipped_lines = 0
        for line in infile:
            line_count += 1
            if not buffer:
                buffer = line.strip()
            else:
                buffer += ' ' + line.strip()
            
            # Check the number of columns
            columns_count = buffer.count(',') + 1
            if columns_count == expected_columns:
                outfile.write(buffer + '\n')
                buffer = ""
            elif columns_count > expected_columns:
                # Too many columns, reset the buffer
                print(f"Skipping malformed line (line {line_count})")
                buffer = ""
                skipped_lines += 1
        
        # Write any remaining buffered line if it has the correct number of columns
        if buffer and buffer.count(',') + 1 == expected_columns:
            outfile.write(buffer + '\n')
        
        print(f"Total lines processed: {line_count}")
        print(f"Total lines skipped: {skipped_lines}")


        
def update_VINs_data_file_maker(cleanedNormalized_resultedData, VINs_data_file_path):
    df = cleanedNormalized_resultedData[pd.to_numeric(cleanedNormalized_resultedData['calendar_day']) > 1804]    
    df = df[['VIN', 'calendar_day']]  #.filter(pd.to_numeric(cleanedNormalized_resultedData['calendar_day']) > 1804) when .filter is used only column inside the filter condition will be selected. 
    df.sort_values(['VIN', 'calendar_day'], ascending=[True, False], inplace=True)   # Also true and works the same:   df = df.sort_values(by=['VIN', 'calendar_day'], ascending=[True, False])

    # Drop duplicates, keeping the first occurrence (which has the maximum side column value due to sorting)
    df = df.drop_duplicates(subset='VIN', keep='first')
    df.columns = ['VIN', 'TOTAL_ROWS']
    # Save the modified DataFrame back to an Excel file
    df.to_csv(VINs_data_file_path, index = None, mode = 'w', header=False)
    return
  
                
'''
This converts a bytes string to a float between 0.0 and 1.0. If we are using a unicode string (e.g., in python 3), then we need to encode it. 
This should give the same result on any machine and any version of python (tested on python 2.7 and 3.5).'''

def is_valid_VIN(vin):
    """
    Validate a Vehicle Identification Number (VIN).
    
    Args:
    vin (str): The VIN to validate.

    Returns:
    bool: True if the VIN is valid, False otherwise.
    """
    if size(vin) != 17:
        return 0  # VIN must be exactly 17 characters
    
    # Check for invalid characters (I, O, Q are not used in VINs to avoid confusion with numbers)
    if any(char in 'IOQ' for char in vin):
        return 0

    # VIN Check digit calculation (position 9)
    transliteration_dict = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5,
        'F': 6, 'G': 7, 'H': 8, 'J': 1, 'K': 2,
        'L': 3, 'M': 4, 'N': 5, 'P': 7, 'R': 9,
        'S': 2, 'T': 3, 'U': 4, 'V': 5, 'W': 6,
        'X': 7, 'Y': 8, 'Z': 9
    }
    weights = [8, 7, 6, 5, 4, 3, 2, 10, 0, 9, 8, 7, 6, 5, 4, 3, 2]
    
    # Calculate the check sum
    check_sum = 0
    for i, char in enumerate(vin):
        if char in transliteration_dict:
            check_sum += transliteration_dict[char] * weights[i]
    
    # Determine the check digit
    check_digit = check_sum % 11
    check_digit = 'X' if check_digit == 10 else str(check_digit)
    
    # Check digit is the 9th character in the VIN
    if vin[8] == check_digit:
        return 1
    else:
        return 0


# Loading Generated Dataframe

In [5]:
input_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv'
cleaned_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv'
VINs_data_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv'
expected_columns = 723  # The correct number of columns


# Preprocess the CSV file to fix bad lines
preprocess_csv(input_file_path, cleaned_file_path, expected_columns)


selected_features_from_population = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s]
all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features


# Read the cleaned CSV file into a DataFrame
cleanedNormalized_resultedData = pd.read_csv(cleaned_file_path, header=None, names=all_columns_names, index_col=False, dtype='unicode')

# Display the DataFrame
print("DataFrame loaded successfully:")
display(cleanedNormalized_resultedData)  #cleanedNormalized_resultedData[['VIN','calendar_day']])
print(f"Number of rows read: {len(cleanedNormalized_resultedData)}")

Skipping malformed line (line 266)
Skipping malformed line (line 390)
Skipping malformed line (line 701)
Skipping malformed line (line 814)
Skipping malformed line (line 821)
Skipping malformed line (line 1523)
Skipping malformed line (line 1843)
Skipping malformed line (line 1845)
Skipping malformed line (line 3245)
Skipping malformed line (line 4002)
Skipping malformed line (line 4578)
Skipping malformed line (line 4824)
Skipping malformed line (line 5426)
Skipping malformed line (line 5614)
Skipping malformed line (line 5680)
Skipping malformed line (line 7150)
Skipping malformed line (line 7233)
Skipping malformed line (line 7313)
Skipping malformed line (line 8229)
Skipping malformed line (line 8770)
Skipping malformed line (line 9688)
Skipping malformed line (line 10298)
Skipping malformed line (line 10413)
Skipping malformed line (line 10606)
Skipping malformed line (line 10716)
Skipping malformed line (line 11549)
Skipping malformed line (line 12972)
Skipping malformed line (li

,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,4V4NC9EH1JN886550,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,1M2AX07C8FM024012,D13,455,GU7,WTDF22.5,WTDD24.5,?,WBF-ALCO,WBD-ALCO,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27777,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27778,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27779,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27780,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


Number of rows read: 27782


# Validating and Cleaning of the Generated Data

In [6]:
# removing bad rows with more cells (columns) in each row than expected.
max_columns = cleanedNormalized_resultedData.shape[1]

cleanedNormalized_resultedData = cleanedNormalized_resultedData[cleanedNormalized_resultedData['VIN'].str.len() == 17]  #cleanedNormalized_resultedData.filter(cleanedNormalized_resultedData['VIN'].str.len() == 17) only selects column VIN

# 1 - Removing any possible leading or trailing whitespace from the column names may cause issues.
cleanedNormalized_resultedData.columns = cleanedNormalized_resultedData.columns.str.strip()

# 2 - removing bad rows with more cells (columns) in each row than expected.
max_columns = cleanedNormalized_resultedData.shape[1]
cleanedNormalized_resultedData = cleanedNormalized_resultedData[cleanedNormalized_resultedData.apply(lambda x: len(x) <= max_columns, axis=1)]

# 3 - fill empty values with 0
cleanedNormalized_resultedData = cleanedNormalized_resultedData.fillna(0)
print(f"Cleaned DataFrame shape: {cleanedNormalized_resultedData.shape}")

# Generage updated VINs_data file
update_VINs_data_file_maker(cleanedNormalized_resultedData, VINs_data_file_path)
# print(cleanedNormalized_resultedData.columns)

# display(len(cleanedNormalized_resultedData['VIN'].unique()))

cleanedNormalized_resultedData.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv', index=None, mode='w', header=False)
display(cleanedNormalized_resultedData)

Cleaned DataFrame shape: (27782, 723)


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,4V4NC9EH1JN886550,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,1M2AX07C8FM024012,D13,455,GU7,WTDF22.5,WTDD24.5,?,WBF-ALCO,WBD-ALCO,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,1M2LR2GC9MM004122,D11,355,LR6,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-HL,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27777,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27778,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27779,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
27780,1M1AN07Y9FM020086,D13,500,CHU,WTDF24.5,WTDD24.5,?,WBF-ALCO,WBD-ACCU,?,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [7]:
print(cleanedNormalized_resultedData.iloc[:, 592])  # col[0] is VIN, col[1] to col[591] KOLA features. col[592] is calendar_day. col[593] to col[720] are calculated features. col[721] to col[722] are target values. 

0        0
1        6
2        0
3        4
4        6
        ..
27202    5
27203    1
27204    3
27205    9
27206    7
Name: calendar_day, Length: 27207, dtype: object


# Example1: None Sparse Data Normalization Example

In [8]:
# column_names = [f'Column_{i}' for i in range(num_columns)]  # Generate column names
# df = pd.DataFrame(data, columns=column_names)
# print("original DataFrame with Random Numeric Values:")
# print(df)
# df = normalize_numeric_feature_values("Column", df)
# # Display the DataFrame
# print("Generated DataFrame with Random Numeric Values:")
# print(df)

# Example2:Sparse Data Normalization Example

In [9]:
# # Example usage
# # Generating a sparse DataFrame with random float values and many zeros
# num_rows = 10
# num_columns = 5
# data = np.random.uniform(0, 1, size=(num_rows, num_columns))
# data[data < 0.7] = 0  # Introduce sparsity by setting 70% of the data to zero
# df = pd.DataFrame(data, columns=[f'Column_{i}' for i in range(num_columns)])

# print("Original DataFrame:")
# print(df)

# # Normalize the DataFrame
# normalized_df = normalize_sparse_dataframe(df)

# print("\nNormalized DataFrame:")
# print(normalized_df)

# Normalizing the Generated Data

In [10]:
# Read VINs data
VINs_data_df = pd.read_csv(VINs_data_file_path, header=None, names=['VIN', 'TOTAL_ROWS'], index_col=False, dtype='unicode')
completed_calc_VINs = VINs_data_df["VIN"].reset_index(drop=True)

# Process the data for each VIN
for i, thisVIN in completed_calc_VINs.items():
    print(thisVIN, i)
    whole_data_for_thisVIN = cleanedNormalized_resultedData.loc[cleanedNormalized_resultedData['VIN'] == thisVIN].copy()

    data_size = len(whole_data_for_thisVIN)

    # 1 - 1. Cleaning numeric chunk of the dataframe
    selected_numeric_features_for_this_VIN = [s for s in all_columns_names if '_dtc' in s] + ['calendar_day']
    for col in selected_numeric_features_for_this_VIN:
        whole_data_for_thisVIN.loc[:, col] = pd.to_numeric(whole_data_for_thisVIN[col])

    # Cleaning non-numeric chunk of the dataframe
    selected_non_numeric_features_for_this_VIN = [s for s in all_columns_names if '_dtc' not in s]
    whole_data_for_thisVIN.loc[:, selected_non_numeric_features_for_this_VIN] = whole_data_for_thisVIN[selected_non_numeric_features_for_this_VIN].replace('', np.nan)

    # Normalizing numeric chunk of the dataframe 
    normalized_numeric_chunk = normalize_sparse_dataframe(whole_data_for_thisVIN[selected_numeric_features_for_this_VIN], "calendar_day")
    whole_data_for_thisVIN.loc[:, selected_numeric_features_for_this_VIN] = normalized_numeric_chunk

    cleanedNormalized_resultedData.loc[cleanedNormalized_resultedData['VIN'] == thisVIN, :] = whole_data_for_thisVIN

# Remove rows with VINs not in completed_calc_VINs
cleanedNormalized_resultedData = cleanedNormalized_resultedData[cleanedNormalized_resultedData['VIN'].isin(completed_calc_VINs)]

cleanedNormalized_resultedData.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv', index=None, mode='w', header=False)
print("Whole data for all VINs: \n")
print(cleanedNormalized_resultedData)

Whole data for all VINs: 

Empty DataFrame
Columns: [VIN, ENGINE_SIZE, ENGINE_HP, VEH_TYPE, _KOLA_01X, _KOLA_02X, _KOLA_03X, _KOLA_04X, _KOLA_05X, _KOLA_06X, _KOLA_07X, _KOLA_08A, _KOLA_08X, _KOLA_09X, _KOLA_0AX, _KOLA_0BX, _KOLA_0CX, _KOLA_0DX, _KOLA_0EA, _KOLA_0EX, _KOLA_0GX, _KOLA_0HA, _KOLA_0HX, _KOLA_0JX, _KOLA_0KX, _KOLA_0LX, _KOLA_0NX, _KOLA_0PA, _KOLA_0PX, _KOLA_0RA, _KOLA_0RX, _KOLA_0SX, _KOLA_15X, _KOLA_19A, _KOLA_1BX, _KOLA_1CX, _KOLA_1DA, _KOLA_1DX, _KOLA_1EA, _KOLA_1GA, _KOLA_1HA, _KOLA_1JX, _KOLA_1KX, _KOLA_1LA, _KOLA_1NX, _KOLA_1PB, _KOLA_1PX, _KOLA_1QB, _KOLA_1RB, _KOLA_1RX, _KOLA_1SB, _KOLA_1SX, _KOLA_1TX, _KOLA_1UX, _KOLA_1VX, _KOLA_1YA, _KOLA_1ZC, _KOLA_20X, _KOLA_21X, _KOLA_24X, _KOLA_26X, _KOLA_28X, _KOLA_2AC, _KOLA_2BC, _KOLA_2DX, _KOLA_2JC, _KOLA_2KX, _KOLA_2MC, _KOLA_2PB, _KOLA_2PX, _KOLA_2WX, _KOLA_2XX, _KOLA_31X, _KOLA_34X, _KOLA_36X, _KOLA_38X, _KOLA_39X, _KOLA_3AX, _KOLA_3FC, _KOLA_3GC, _KOLA_3GX, _KOLA_3HA, _KOLA_3HX, _KOLA_3IA, _KOLA_3MB, _KOLA_3NC, _KOLA_